# Límite superior para una variable de Poisson

Esta demo calcula el límite superior del parámetro μ de una distribución de Poisson

Definimos el nivel de confianza del límite superior:

In [ ]:
confidence_level = 0.9

Y la probabilidad de la cola inferior de la distribución:

In [ ]:
tail_probability = 1 - confidence_level

Consideramos un rango de valores observados de la variable aleatoria de Poisson $k \sim \mathrm{Poisson}(\mu)$:

In [ ]:
import numpy as np
kmax = 10
k = np.arange(0, kmax+1)

Para cada valor posible $k_{obs}$, calculamos el valor de $\mu$ tal que $P(k \le k_{obs}) = 1 - \mathrm{CL}$: 

In [ ]:
from scipy.stats import poisson
from scipy.optimize import brentq

upper_limit = []

for k_i in k:
   
    # CDF complement minus the target confidence level 
    def f(mu):
        # fixed random variable and free parameter mu as in a likelihood function
        return poisson.cdf(k_i, mu) - tail_probability

    # Searching the value of mu that sets the CDF complement equal to the target confidence level
    mu_i =  brentq(f, a=0, b=100)

    # This is the upper limit for the observed k_i
    upper_limit.append(mu_i)

Como en este problema la variable aleatoria es discreta, es más práctico calcular el cinturón de confianza en un solo paso en lugar del procedimiento de dos pasos que usamos hasta ahora. Los límites toman valores discretos como la variable aleatoria aunque no son valores enteros. La siguiente figura muestra la función de distribución acumulada para uno de los límites superiores de $\mu$ encontrados:  

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_xlabel("$k$")
ax.set_ylabel("$F(k)$")

# Select event to plot
k_obs = 1
mu_up = upper_limit[k_obs]

# Plot CDF
cumulative_probability = poisson.cdf(k, mu=mu_up)
ax.scatter(k, cumulative_probability)

# Set axis limits to control the positioning of my text  
ymin = -0.05
ax.set_ylim(bottom=ymin)
xmin = -0.5
ax.set_xlim(left=xmin)

# Plot μ line 
ax.axvline(mu_up, ls='--', color='tab:green')
ax.text(mu_up, 0.25,"   $\mu_{up}$", transform = ax.get_xaxis_transform(), color='tab:green', fontsize='large')

# Plot observed event coordinates
ax.vlines(x=k_obs, ymin=ymin, ymax=tail_probability, ls='--', color='tab:orange')
ax.text(k_obs, ymin,"   $k_{obs}$", ha='center', va='top', color='tab:orange', fontsize='large')
ax.hlines(y=tail_probability, xmin=xmin, xmax=k_obs, ls='--', color='tab:orange')
ax.text(xmin, tail_probability,"$1 - CL$  ", ha='right', va='center', color='tab:orange', fontsize='large')

ax.text(0.7, 0.5, f"CL = {confidence_level} \n $k_{{obs}} = {k_obs}$ \n $\mu_{{up}}$ = {mu_up:.2f}", transform=ax.transAxes)


fig.savefig("limite_superior.svg")

Vemos que para este valor $\mu_{up}$ la probabilidad que k sea menor o igual al dato observado es igual a 1 menos el nivel de confianza. Este es justamente el criterio que usamos para encontrar $\mu_{up}$. 

Mostramos los límites superiores de $\mu$ para cada $k$ observado:

In [ ]:
import pandas as pd
df = pd.DataFrame({"$\mu_{up}$": upper_limit}, index=k)
df.index.name = "k"
df

Ploteamos el cinturón de confianza

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_xlabel("$\mu$")
ax.set_ylabel("$k$")
mu = np.linspace(0, upper_limit[-1], 100)
ax.plot(mu, mu, label="Mean")
ax.plot(upper_limit, k, ls='', marker='o', label="$\mu_{up}$")
ax.set_xlim(left=0)
ax.legend()

El cinturón de confianza es discreto al igual que la variable $k$.

El caso más frecuente de uso del límite superior es en un experimento que se observan cero eventos, $k=0$. En este caso el límite superior del parámetro de Poisson es $\mu_{up} = 2.30$. En la práctica se usa un límite más laxo $\mu_{up} = 2.44$ que unifica el método para calcular intervalos de confianza y límites. Más detalles sobre esto en: [G. J. Feldman and R. D. Cousins, Phys. Rev. D57, 3873 (1998)]( 	
https://doi.org/10.1103/PhysRevD.57.3873).  